In [2]:
import os
import json
import warnings
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from peft import PeftModel, PeftConfig

import bentoml


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Seoul')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:

# T5

In [ ]:
# PEFT_MODEL_ID = 'checkpoints/paust_pko_t5_base_v4_run_1'

# peft_config = PeftConfig.from_pretrained(PEFT_MODEL_ID)

# model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path)
# model = PeftModel.from_pretrained(model, PEFT_MODEL_ID)

# model = model.merge_and_unload()

# model.save_pretrained('checkpoints/paust_pko_t5_base_v4_run_1/merged')

# Save bento model

In [4]:
# model = AutoModelForSeq2SeqLM.from_pretrained('checkpoints/paust_pko_t5_base_v4_run_1/merged')

In [5]:
# bento_model = bentoml.transformers.save_model(
#     "news-topic-keyphrase-generator-t5-base",
#     model,
#     signatures={
#         "generate": {"batchable": True},
#     },
# )
# bento_model

Model(tag="news-topic-keyphrase-generator-t5-base:iuqguuhzd6pfwasc", path="/root/bentoml/models/news-topic-keyphrase-generator-t5-base/iuqguuhzd6pfwasc/")

In [6]:
# load bento model

runner = bentoml.transformers.get("news-topic-keyphrase-generator-t5-base:iuqguuhzd6pfwasc").to_runner()

In [7]:
runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [8]:
PEFT_MODEL_ID = 'checkpoints/paust_pko_t5_base_v4_run_1'
tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL_ID)

In [13]:
def news_topic_keyphrase_generator_t5_base(input_texts_json):
    prefix = "generate keyphrases: "

    inputs = json.loads(input_texts_json)["inputs"]
    inputs = [prefix + input_ for input_ in inputs]
    encoded = tokenizer(
        inputs,
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    outputs = runner.generate.run(**encoded, max_length=64).detach().cpu().tolist()
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    outputs = dict(outputs=outputs)
    outputs = json.dumps(outputs)

    return outputs


In [5]:
test_case_1 = """'결정만 남은' 맨유, 김민재 바이아웃 발동 고려 중 [포포투=백현기]맨체스터 유나이티드는 김민재 영입에 진심이다.나폴리는 리그 우승을 눈앞에 두고 있다. 나폴리는 이번 시즌 스쿠데토에 가까워져 있다. 현재 리그 28경기를 치른 시점에서 승점 71점으로 선두를 기록하고 있다. 나폴리는 디에고 마라도나가 군림했던 1989-90시즌 이후 첫 우승이 임박했다. 또한 나폴리는 유럽축구연맹(UEFA) 챔피언스리그(UCL) 8강에 진출해 있으며 구단 역사상 최고 성적을 거두고 있다.나폴리의 고공 행진의 중심에 수비수 김민재가 있다. 김민재는 이번 시즌 '역대급 시즌'을 보내고 있다. 데뷔 시즌이라는 사실이 무색할 만큼 세리에 A에서의 첫 시즌에서 안정적인 수비력을 보여주고 있다. 특히 김민재의 안정적인 수비력과 넓은 커버 범위 덕분에 나폴리는 마음 놓고 공격 라인을 더 높이 올릴 수 있다.첫 시즌부터 세리에 A에 연착륙해 맹활약하고 있는 김민재에게 극찬도 쏟아졌다. 지난 1월에는 AS 로마의 조세 무리뉴 감독이 "나폴리 선수 중 한 명에 대해 얘기할 말이 있다. 과거 토트넘에 있을 당시 김민재와 화상 통화도 했을 정도로 그를 영입하고 싶었다. 하지만 토트넘은 나를 도와주지 않았다"라고 김민재를 극찬했다.이어 루치아노 스팔레티 감독도 김민재를 아끼고 있다. 김민재의 수비력과 커버 범위, 축구 지능을 믿고 나폴리는 더 공격적인 축구 구현이 가능해졌다. 스팔레티 감독은 핵심이 된 김민재에게 "그는 내게 있어 세계 최고의 수비수"라는 극찬을 쏟아내기도 했다.자연스럽게 빅클럽들의 관심을 받고 있다. 가장 적극적인 구단은 맨체스터 유나이티드다. 유럽축구 이적시장 전문가 파브리시오 로마노에 따르면, 맨체스터 유나이티드는 김민재를 관찰하기 위해 이미 스카우터를 파견했으며 꾸준하게 관찰 중이라 전해진다.김민재의 이적의 가장 큰 변수는 바로 상대적으로 낮은 바이아웃 금액이다. 영국 매체 '데일리 메일'에 따르면, 김민재는 올 여름 일시적으로 4500만 유로(약 646억 원)의 바이아웃 조항을 가지고 있다. 김민재의 시장 가치에 비해 상대적으로 낮은 바이아웃 금액에 나폴리는 김민재와의 새 계약을 시즌 내내 추진하고 있다.맨유는 이 바이아웃에 집중하고 있다. '데일리 메일'은 "맨유는 김민재의 바이아웃 조항을 발동하는 것을 고려 중이다"고 밝혔다. 매체는 이어 "리버풀과 토트넘 훗스퍼 또한 김민재 영입을 위해 경쟁하고 있다"고 덧붙이면서 잉글랜드 클럽들의 관심을 소개하기도 했다.기사제공 포포투백현기 기자 hkbaek11@fourfourtwo.co.krCopyright c 포포투. All rights reserved. 무단 전재 및 재배포 금지.기사 섹션 분류 가이드포포투 FourFourTwo 잡지 정기 구독하기"""
test_case_2 = """HDC, 광주 화정동 붕괴 아파트 해체공사 착수...2025년 철거 완료 HDC현대산업개발(HDC)이 붕괴 사고가 일어난 광주화정 아이파크의 전면 해제공사에 돌입했다. 오는 2025년 상반기 철거 작업을 마칠 전망이다.HDC현대산업개발 A1 추진단은 6일 A1 현장 해체공사를 위한 관계기관의 인허가 승인 및 공사 중지 해제 등 행정절차가 완료되면서 해체 작업을 시작한다고 밝혔다.외벽 안정화 작업을 마치고 본격 해체를 앞둔 광주화정 아이파크 A1현장 전경[사진제공=HDC현대산업개발]HDC는 지난 1월 16일 201동의 안정화 작업을 마친 후 약 2개월 동안 해체계획·안전관리계획·유해위험방지계획 등의 인허가를 위한 행정절차를 진행해 온 광주광역시 서구청 등 관계기관과 해체공사가 안전하게 진행될 수 있도록 철저하게 계획을 검토하고 보완해 왔다.이번 해체공사 계획은 유례없는 것으로 준공을 앞두고 있던 고층 건물의 해체 작업인데다 현장이 도심부에 위치해 안전사고와 환경피해 예방에 초점을 맞추고 있다. 해체공사는 약 2년간 진행돼 2025년 상반기 중 완료할 예정이다.HDC현대산업개발은 먼저 구조물 철거공사를 진행하기 위한 준비작업에 들어간다. 준비작업은 압쇄 등 본격적인 해체 작업 전에 선행되어야 하는 작업으로 시스템 비계, RCS(Rail Climbing System), 타워크레인, 호이스트 카 등의 가시설 설치와 내부 마감재를 해체하는 작업을 말한다. 2~3개월 정도의 준비작업 후 본격적인 구조물 철거작업은 6월 중순부터 시작할 예정이다.A1추진단은 해체공사를 진행하기에 앞서 지난 3월 29일 기술자문단 회의를 HDC현대산업개발 본사에서 개최했다. 이날 회의에는 최창식 한양대 교수를 단장으로 하는 기술자문단과 A1추진단장을 포함한 A1추진단원, 본사 관계자들이 참석했다. 이번 기술자문단 회의에서 A1추진단이 해체계획을 발표한 후 기술자문단과 구조, 안전, 환경 등 분야별 관리방안에 대해 의견을 나누면서 해체계획 전반에 대해 재점검했다.호명기 A1추진단장은 “국내외 전문가 자문 등을 통해 구조, 안전에 대한 검토와 기술 보완을 진행해 왔다”며 “해체계획 인허가에 다소 시간이 소요되었지만 철저하게 준비한 만큼 계획대로 해체공사를 안전하게 진행하고, 지역사회 관계자들과 소통하면서 신뢰를 회복해 나갈 것”이라고 밝혔다."""

input_texts = dict(
    inputs = [test_case_1, test_case_2]
)
print(input_texts)
input_texts_json = json.dumps(input_texts)
# input_texts_json

{'inputs': ['\'결정만 남은\' 맨유, 김민재 바이아웃 발동 고려 중 [포포투=백현기]맨체스터 유나이티드는 김민재 영입에 진심이다.나폴리는 리그 우승을 눈앞에 두고 있다. 나폴리는 이번 시즌 스쿠데토에 가까워져 있다. 현재 리그 28경기를 치른 시점에서 승점 71점으로 선두를 기록하고 있다. 나폴리는 디에고 마라도나가 군림했던 1989-90시즌 이후 첫 우승이 임박했다. 또한 나폴리는 유럽축구연맹(UEFA) 챔피언스리그(UCL) 8강에 진출해 있으며 구단 역사상 최고 성적을 거두고 있다.나폴리의 고공 행진의 중심에 수비수 김민재가 있다. 김민재는 이번 시즌 \'역대급 시즌\'을 보내고 있다. 데뷔 시즌이라는 사실이 무색할 만큼 세리에 A에서의 첫 시즌에서 안정적인 수비력을 보여주고 있다. 특히 김민재의 안정적인 수비력과 넓은 커버 범위 덕분에 나폴리는 마음 놓고 공격 라인을 더 높이 올릴 수 있다.첫 시즌부터 세리에 A에 연착륙해 맹활약하고 있는 김민재에게 극찬도 쏟아졌다. 지난 1월에는 AS 로마의 조세 무리뉴 감독이 "나폴리 선수 중 한 명에 대해 얘기할 말이 있다. 과거 토트넘에 있을 당시 김민재와 화상 통화도 했을 정도로 그를 영입하고 싶었다. 하지만 토트넘은 나를 도와주지 않았다"라고 김민재를 극찬했다.이어 루치아노 스팔레티 감독도 김민재를 아끼고 있다. 김민재의 수비력과 커버 범위, 축구 지능을 믿고 나폴리는 더 공격적인 축구 구현이 가능해졌다. 스팔레티 감독은 핵심이 된 김민재에게 "그는 내게 있어 세계 최고의 수비수"라는 극찬을 쏟아내기도 했다.자연스럽게 빅클럽들의 관심을 받고 있다. 가장 적극적인 구단은 맨체스터 유나이티드다. 유럽축구 이적시장 전문가 파브리시오 로마노에 따르면, 맨체스터 유나이티드는 김민재를 관찰하기 위해 이미 스카우터를 파견했으며 꾸준하게 관찰 중이라 전해진다.김민재의 이적의 가장 큰 변수는 바로 상대적으로 낮은 바이아웃 금액이다. 영국 매체 \'데일리 메일\'에 따르면, 김민재는 올 여름 일시적으로 4500만 유

In [ ]:
input_texts_json.

In [19]:
outputs = news_topic_keyphrase_generator_t5_base(input_texts_json)
outputs

'{"outputs": ["\\ub9e8\\uccb4\\uc2a4\\ud130 \\uc720\\ub098\\uc774\\ud2f0\\ub4dc; \\uae40\\ubbfc\\uc7ac; \\ubc14\\uc774\\uc544\\uc6c3; \\ub9ac\\uadf8 \\uc6b0\\uc2b9; \\uc2a4\\ucfe0\\ub370\\ud1a0; \\uc2b9\\uc810 71\\uc810; \\ub514\\uc5d0\\uace0 \\ub9c8\\ub77c\\ub3c4\\ub098; UEFA \\ucc54\\ud53c\\uc5b8\\uc2a4\\ub9ac\\uadf8; \\uc218\\ube44\\ub825; \\ucee4\\ubc84 \\ubc94\\uc704", "HDC\\ud604\\ub300\\uc0b0\\uc5c5\\uac1c\\ubc1c; \\ubd95\\uad34 \\uc0ac\\uace0; \\uad11\\uc8fc\\ud654\\uc815 \\uc544\\uc774\\ud30c\\ud06c; \\ud574\\uccb4\\uacf5\\uc0ac; 2025\\ub144 \\uc0c1\\ubc18\\uae30; \\ucca0\\uac70 \\uc791\\uc5c5; \\uad00\\uacc4\\uae30\\uad00; \\uc778\\ud5c8\\uac00 \\uc2b9\\uc778; \\uc548\\uc804\\uad00\\ub9ac\\uacc4\\ud68d; \\uc720\\ud574\\uc704\\ud5d8\\ubc29\\uc9c0\\uacc4\\ud68d"]}'

In [20]:
outputs = json.loads(outputs)
outputs

{'outputs': ['맨체스터 유나이티드; 김민재; 바이아웃; 리그 우승; 스쿠데토; 승점 71점; 디에고 마라도나; UEFA 챔피언스리그; 수비력; 커버 범위',
  'HDC현대산업개발; 붕괴 사고; 광주화정 아이파크; 해체공사; 2025년 상반기; 철거 작업; 관계기관; 인허가 승인; 안전관리계획; 유해위험방지계획']}

In [ ]:
# 2023-05-23T13:25:53+0900 [ERROR] [runner:news-topic-keyphrase-generator-t5-base:1] Traceback (most recent call last):
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner.py", line 297, in init_local
#     self._set_handle(LocalRunnerRef)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner.py", line 140, in _set_handle
#     runner_handle = handle_class(self, *args, **kwargs)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner_handle/local.py", line 24, in __init__
#     self._runnable = runner.runnable_class(**runner.runnable_init_params)  # type: ignore
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/frameworks/transformers.py", line 901, in __init__
#     self.model = load_model(bento_model, **kwargs)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/frameworks/transformers.py", line 458, in load_model
#     return protocol.from_pretrained(bento_model.path, **kwargs)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/transformers/modeling_utils.py", line 2611, in from_pretrained
#     model = cls(config, *model_args, **model_kwargs)
# TypeError: __init__() got an unexpected keyword argument 'device'

# 2023-05-23T13:25:53+0900 [ERROR] [runner:news-topic-keyphrase-generator-t5-base:1] Traceback (most recent call last):
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/starlette/routing.py", line 671, in lifespan
#     async with self.lifespan_context(app):
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/starlette/routing.py", line 566, in __aenter__
#     await self._router.startup()
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/starlette/routing.py", line 650, in startup
#     handler()
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner.py", line 307, in init_local
#     raise e
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner.py", line 297, in init_local
#     self._set_handle(LocalRunnerRef)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner.py", line 140, in _set_handle
#     runner_handle = handle_class(self, *args, **kwargs)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/runner/runner_handle/local.py", line 24, in __init__
#     self._runnable = runner.runnable_class(**runner.runnable_init_params)  # type: ignore
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/frameworks/transformers.py", line 901, in __init__
#     self.model = load_model(bento_model, **kwargs)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bentoml/_internal/frameworks/transformers.py", line 458, in load_model
#     return protocol.from_pretrained(bento_model.path, **kwargs)
#   File "/workspace/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/transformers/modeling_utils.py", line 2611, in from_pretrained
#     model = cls(config, *model_args, **model_kwargs)
# TypeError: __init__() got an unexpected keyword argument 'device'